<a href="https://colab.research.google.com/github/TetraPlex-org/audiomerge/blob/main/AUDIOMERGE_FD_20230807.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Boilerplate

In [2]:
!git clone https://github.com/TetraPlex-org/audiomerge.git

Cloning into 'audiomerge'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 24 (delta 3), reused 6 (delta 0), pack-reused 11
Receiving objects: 100% (24/24), 87.45 MiB | 22.98 MiB/s, done.
Resolving deltas: 100% (3/3), done.


In [3]:
from operator import getitem
from pathlib import Path

import numpy as np
import pywt
import pandas as pd
from scipy import ndimage, stats, signal, integrate, interpolate
from scipy.signal import stft
from scipy.io import wavfile
import pywt
from skimage.restoration import denoise_wavelet

# for quick sketches
from matplotlib import pyplot as plt
from matplotlib.pyplot import plot, hlines, vlines, semilogy

# for professional visualization, publication quality
import plotnine as p9
from plotnine import ggplot, aes, element_blank

# initialization
from IPython.core.display import display, HTML
display(
    HTML(
        "<style>.container { width:80%; transform: translateX(130px); !important; }</style>"
    )
)
plt.rc("figure", figsize=(20, 20))

Pipes are awesome. Let's do more of those.

In [4]:
import ast
import inspect
from itertools import takewhile
from textwrap import dedent

class _PipeTransformer(ast.NodeTransformer):
    def visit_BinOp(self, node):
        if not isinstance(node.op, (ast.LShift, ast.RShift)):
            return node
        if not isinstance(node.right, ast.Call):
            return self.visit(
                ast.Call(
                    func=node.right,
                    args=[node.left],
                    keywords=[],
                    starargs=None,
                    kwargs=None,
                    lineno=node.right.lineno,
                    col_offset=node.right.col_offset,
                )
            )
        node.right.args.insert(0 if isinstance(node.op, ast.RShift) else len(node.right.args), node.left)
        return self.visit(node.right)

def pipes(func_or_class):
    if inspect.isclass(func_or_class):
        decorator_frame = inspect.stack()[1]
        ctx = decorator_frame[0].f_locals
        first_line_number = decorator_frame[2]
    else:
        ctx = func_or_class.__globals__
        first_line_number = func_or_class.__code__.co_firstlineno
    source = inspect.getsource(func_or_class)
    tree = ast.parse(dedent(source))
    ast.increment_lineno(tree, first_line_number - 1)
    source_indent = sum(1 for _ in takewhile(str.isspace, source)) + 1
    for node in ast.walk(tree):
        if hasattr(node, "col_offset"):
            node.col_offset += source_indent
    tree.body[0].decorator_list = [
        d
        for d in tree.body[0].decorator_list
        if isinstance(d, ast.Call) and d.func.id != "pipes" or isinstance(d, ast.Name) and d.id != "pipes"
    ]
    tree = _PipeTransformer().visit(tree)
    code = compile(tree, filename=(ctx["__file__"] if "__file__" in ctx else "repl"), mode="exec")
    exec(code, ctx)
    return ctx[tree.body[0].name]

def pipeprint(*args, **kwargs):
    print(*args, **kwargs)
    return args[0]

def convert_to_float(A):
    match A.dtype:
        case "int16":
            return A/(2**(16-1)-1)
        case "float64":
            # normalize to [-1, 1]
            A = (A - np.min(A)) / (np.max(A) - np.min(A))
            A = 2 * A - 1
            return A
        case _:
            raise NotImplementedError

Experiment

In [5]:
rate_A, data_A = wavfile.read("audiomerge/examples/1/A.wav")
rate_B, data_B = wavfile.read("audiomerge/examples/1/B.wav")
rate_C, data_C = wavfile.read("audiomerge/examples/1/C.wav")
rate_D, data_D = wavfile.read("audiomerge/examples/1/D.wav")
rate_ref, data_ref = wavfile.read("audiomerge/examples/1/reference.wav")

data_A = convert_to_float(data_A)
data_B = convert_to_float(data_B)
data_C = convert_to_float(data_C)
data_D = convert_to_float(data_D)
data_ref = convert_to_float(data_ref)